## Вступление

Анализ базы данных сервиса для чтения книг по подписке

Заказчик: сервис для чтения книг

Цель анализа: анализ базы данных приобретенного сервиса для создания концепции нового продукта

## Подключение к бд

In [59]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': '***********', # имя пользователя
'pwd': '****', # пароль
'host': '********',
'port': ****, # порт подключения
'db': '**********'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Знакомство с таблицами

In [60]:
query = '''
SELECT *
FROM books
LIMIT 5'''

pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [61]:
query = '''
SELECT *
FROM authors
LIMIT 5'''

pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [62]:
query = '''
SELECT *
FROM publishers
LIMIT 5'''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [63]:
query = '''
SELECT *
FROM ratings
LIMIT 5'''

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [64]:
query = '''
SELECT *
FROM reviews
LIMIT 5'''

pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Первая задача

In [65]:
# Посчитайте, сколько книг вышло после 1 января 2000 года;
query = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'
'''

pd.io.sql.read_sql(query, con = engine)

,count
0,819


819 книг и брошюр было добавлено после 1 января 2000 года

## Вторая задача

In [66]:
# Для каждой книги посчитайте количество обзоров и среднюю оценку;
query = '''
SELECT b.book_id ,b.title, COUNT(DISTINCT rv.review_id), AVG(rt.rating)
FROM books b
LEFT JOIN reviews rv on b.book_id=rv.book_id
LEFT JOIN ratings rt on b.book_id=rt.book_id
GROUP BY 1, 2
ORDER BY 3 DESC
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,title,count,avg
0,948,Twilight (Twilight #1),7,3.662500
1,963,Water for Elephants,6,3.977273
2,734,The Glass Castle,6,4.206897
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...,...
995,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,808,The Natural Way to Draw,0,3.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,221,Essential Tales and Poems,0,4.000000


In [67]:
# тут я усомнился в общем количестве книг и брошюр, проблема была в join
query = '''
select count(book_id)
from books
'''
pd.io.sql.read_sql(query, con = engine)

,count
0,1000


В каталоге 1000 книг, количество обзоров от 7 до 0 (изначально без LEFT JOIN у меня не отображались книги с 0 обзорами).

Средняя оценка не зависит от количества обзоров, скорее всего из-за предпочтений читателей.

## Третья задача

In [68]:
# Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
query = '''
SELECT p.publisher, COUNT(b.book_id)
FROM publishers p
JOIN books b on p.publisher_id=b.publisher_id
WHERE b.num_pages > 50
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1
'''
pd.io.sql.read_sql(query, con = engine)

,publisher,count
0,Penguin Books,42


Больше всего книг толще 50 страниц представлено изданием Penguin Books.

## Четвертая задача

In [69]:
# Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
query = '''
WITH rt AS (SELECT book_id
FROM ratings
GROUP BY 1
HAVING COUNT(rating_id) > 50)

SELECT a.author, AVG(r.rating)
FROM books b
join rt on b.book_id=rt.book_id
join authors a on a.author_id=b.author_id
join ratings r on rt.book_id=r.book_id
GROUP BY 1
ORDER BY 2 DESC
limit 1

'''
pd.io.sql.read_sql(query, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


Самый популярный автор - J.K. Rowling/Mary GrandPré, средняя оценка 4.28 из возможных 5 баллов.

Учтены только те авторы, чьи книги с 50 и более оценками.

## Пятая задача

In [70]:
# Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.
query = '''
SELECT COUNT(DISTINCT rv.review_id) / COUNT(DISTINCT rv.username) avg_reviews
FROM ratings r
join reviews rv on r.username=rv.username
WHERE r.username in (SELECT username
                      FROM ratings
                      GROUP BY username
                      HAVING COUNT(rating_id) > 50)

'''
pd.io.sql.read_sql(query, con = engine)

,avg_reviews
0,24


Среднее количество обзоров пользователей которые поставили больше 50 оценок - 24.